In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader,Dataset
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [3]:
def simple_cnn():
    model = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(32 * 8 * 8, 4)  # 4 classes for rotation
    )
    return model

In [4]:
def rotate_image(x):
    angles = [0, 90, 180, 270]
    angle = torch.randint(0, 4, (1,), dtype=torch.int64).item()
    return transforms.functional.rotate(x, angles[angle]), angle

In [5]:
class CustomRotatedCIFAR10(Dataset):
    def __init__(self, original_dataset):
        self.dataset = original_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img, _ = self.dataset[index]
        rotated_img, label = rotate_image(img)
        return rotated_img, label

In [6]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

original_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
rotated_dataset = CustomRotatedCIFAR10(original_dataset)
train_loader = DataLoader(rotated_dataset, batch_size=32, shuffle=True)

Files already downloaded and verified


In [7]:
model = simple_cnn()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [20]:
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = torch.stack([image for image in images])  # Stack images into a single tensor
        labels = torch.tensor(labels)  # Convert labels to a tensor

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

print('Finished Training')

# save the model
torch.save(model.state_dict(), 'rotation_model.pth')

C:\Users\mushfika_rahman1\AppData\Local\Temp\ipykernel_10552\1411649858.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)  # Convert labels to a tensor


Epoch [1/5], Step [100/1563], Loss: 1.3036
Epoch [1/5], Step [200/1563], Loss: 1.2039
Epoch [1/5], Step [300/1563], Loss: 1.1686
Epoch [1/5], Step [400/1563], Loss: 1.1820
Epoch [1/5], Step [500/1563], Loss: 1.1551
Epoch [1/5], Step [600/1563], Loss: 1.1260
Epoch [1/5], Step [700/1563], Loss: 1.1189
Epoch [1/5], Step [800/1563], Loss: 1.1057
Epoch [1/5], Step [900/1563], Loss: 1.0804
Epoch [1/5], Step [1000/1563], Loss: 1.0986
Epoch [1/5], Step [1100/1563], Loss: 1.0784
Epoch [1/5], Step [1200/1563], Loss: 1.0790
Epoch [1/5], Step [1300/1563], Loss: 1.0450
Epoch [1/5], Step [1400/1563], Loss: 1.0521
Epoch [1/5], Step [1500/1563], Loss: 1.0606
Epoch [2/5], Step [100/1563], Loss: 1.0406
Epoch [2/5], Step [200/1563], Loss: 1.0568
Epoch [2/5], Step [300/1563], Loss: 1.0166
Epoch [2/5], Step [400/1563], Loss: 1.0101
Epoch [2/5], Step [500/1563], Loss: 1.0332
Epoch [2/5], Step [600/1563], Loss: 1.0250
Epoch [2/5], Step [700/1563], Loss: 1.0119
Epoch [2/5], Step [800/1563], Loss: 1.0548
Epoch

In [8]:
rotation_model = simple_cnn()
rotation_model.load_state_dict(torch.load('rotation_model.pth'))

<All keys matched successfully>

In [9]:
rotation_model[-1] = nn.Linear(32 * 8 * 8, 10)

In [10]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [14]:
num_epochs = 10
train_losses = []
train_accuracies = []

rotation_model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = rotation_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0
            
    accuracy = 100*total_correct / total_samples
    train_accuracies.append(accuracy)
    train_losses.append(loss.item())

    print(f'Current epoch [{epoch+1}/{num_epochs}], accuracy: {accuracy:.4f}')
    
print('Finished Training on original labels')
# Save the model
torch.save(rotation_model.state_dict(), 'cifar10_classifier_from_rotation.pth')

Epoch [1/10], Step [100/391], Loss: 2.5605
Epoch [1/10], Step [200/391], Loss: 2.5610
Epoch [1/10], Step [300/391], Loss: 2.5647
Current epoch [1/10], accuracy: 9.6040
Epoch [2/10], Step [100/391], Loss: 2.5705
Epoch [2/10], Step [200/391], Loss: 2.5567
Epoch [2/10], Step [300/391], Loss: 2.5455
Current epoch [2/10], accuracy: 9.7140
Epoch [3/10], Step [100/391], Loss: 2.5521
Epoch [3/10], Step [200/391], Loss: 2.5537
Epoch [3/10], Step [300/391], Loss: 2.5619
Current epoch [3/10], accuracy: 9.7340
Epoch [4/10], Step [100/391], Loss: 2.5630
Epoch [4/10], Step [200/391], Loss: 2.5507
Epoch [4/10], Step [300/391], Loss: 2.5507
Current epoch [4/10], accuracy: 9.6360
Epoch [5/10], Step [100/391], Loss: 2.5493
Epoch [5/10], Step [200/391], Loss: 2.5640
Epoch [5/10], Step [300/391], Loss: 2.5603
Current epoch [5/10], accuracy: 9.6200
Epoch [6/10], Step [100/391], Loss: 2.5686
Epoch [6/10], Step [200/391], Loss: 2.5605
Epoch [6/10], Step [300/391], Loss: 2.5493
Current epoch [6/10], accuracy:

In [26]:
#rotation_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        outputs = rotation_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the CIFAR-10 train images: {accuracy:.2f}%')

Accuracy of the model on the CIFAR-10 train images: 12.22%


In [27]:
#rotation_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = rotation_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the CIFAR-10 test images: {accuracy:.2f}%')

Accuracy of the model on the CIFAR-10 test images: 12.44%


In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training Loss")
plt.plot(train_losses, label=" Trainning Loss")
plt.xlabel("Epochs")
plt.ylabel("Trainning Loss")
plt.legend()
plt.show()